In [4]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

# If the model is not available, proceed with the download
llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    # model_id="tiiuae/falcon-7b-instruct",
    task="text-generation",
    # device=0, # 0 == GPU , -1 == CPU , IF 0 is not available on MacBook
    pipeline_kwargs={"max_new_tokens": 150},
)

chain = prompt | llm

chain.invoke({"word": "Chocolate Pie"})

Using pad_token, but it is not set yet.
--- Logging error ---
Traceback (most recent call last):
  File "d:\Project\SangmooGPT\env\lib\site-packages\langchain\callbacks\manager.py", line 2072, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "d:\Project\SangmooGPT\env\lib\site-packages\langchain\callbacks\tracers\langchain.py", line 97, in __init__
    self.client = client or get_client()
  File "d:\Project\SangmooGPT\env\lib\site-packages\langchain\callbacks\tracers\langchain.py", line 55, in get_client
    _CLIENT = Client()
  File "d:\Project\SangmooGPT\env\lib\site-packages\langsmith\client.py", line 272, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "d:\Project\SangmooGPT\env\lib\site-packages\langsmith\client.py", line 181, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must be provided when using hosted LangSmith API

During handling of the above exc

" very old dessert, because it's hard to explain. We just use old chocolate to make it to be chocolate pie again, but you need a very special kind of chocolate to do this.\n\nBakaloo, which is so popular in our village, is a traditional holiday for many who live in North London. Even though we have some shops offering their own bakery on the outskirts of London this year, there is always something to be said about their bakery, which means that we have to be careful about our dessert choices. The bakery will usually take us into an exclusive kitchen with no windows because most of the food is served within ten metres away. However, the shop has the option of opening on Sundays where the baking is done by special"